In [223]:
%pylab widget
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [224]:
# from analysis_functions import *
import fastcat.fastcat as fc

In [225]:
from scipy.optimize import minimize

# Looking at the bowtie filter

In [226]:
# Now lets look for one of the bowtie air projections
bowtie_volume = np.load('/home/xcite/MV_CBCT/projections/kv_air.npy')

plt.figure()
plt.imshow(np.mean(bowtie_volume,0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [227]:
plt.figure()
plt.plot(np.mean(np.mean(bowtie_volume,0),0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
# Should I make an actual fastCAT aluminum model to see what CsI setting agrees with this?
# First I will see if I can make a small model that is just a brick of aluminum

In [5]:
plt.figure()
from scipy.signal import gaussian
lengths3 = gaussian(2*512,8*128/1.5)
plt.plot(1.9 - 2*lengths3**(9))
xx = np.arange(1024)
plt.plot(1.9/(1 + np.e**(-0.02*(xx-512-256))))
plt.plot(np.mean(bowtie_volume[0],0)/20000)
plt.plot((-np.log(np.mean(bowtie_volume[0],0)/20000)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
from scipy.signal import gaussian
lengths = gaussian(512,128/1.5)*2

In [19]:
s = fc.calculate_spectrum(100, 14, 3, 100,monitor=None)
s.attenuate(0.4,fc.get_mu(z=13))
s.attenuate(0.89,fc.get_mu(z=22))
s.x,s.y = s.get_points()

In [20]:
plt.figure()
plt.plot(s.x,s.y)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
edep = np.load('../../fastcat/data/Detectors/CsI-010-micrometer/EnergyDeposition.npy')

In [22]:
plt.figure()
plt.semilogx(edep[1],edep[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
np.sum(s.x)/2

3549.8050000000003

In [72]:
np.argmin(np.abs(np.cumsum(s.y)- np.sum(s.y)/2))

90

In [26]:
s.x[90]

68.64646464646465

## Now we know that the mean energy of the 100 kVp beam should be about 44.6

In [73]:
al = np.loadtxt('../../fastcat/data/mu/14.csv',delimiter=',')

In [74]:
al[0,16]

60.0

In [75]:
mu = (al[1,16])

print(mu)

0.747231


\begin{equation}
y = e^{-\mu x}
\end{equation}

\begin{equation}
-ln(y)/\mu = x
\end{equation}

In [213]:
lengths = -4*np.log(np.mean(np.mean(bowtie_volume[:,250:450],0),0)/21000)/mu

In [228]:
plt.figure()
# lengths2= concatenate((lengths[:512],lengths[511::-1]))*1
plt.plot(lengths)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [215]:
# cut = 50

# xx = np.arange(len(lengths2))

# cut_lengths = np.interp(np.linspace(cut,len(xx)-cut,len(xx)),xx,lengths2)

# figure();plot(cut_lengths)

In [216]:
# dist = np.linspace(-256*0.0784 - 0.0392,256*0.0784 - 0.0392, 512) # TODO: fix this gore!!
# step = 0.05
# dist2 = np.linspace(-256*step - step/2,256*step - step/2, 512) # TODO: fix this gore!!

# xx = np.arange(len(lengths2))

# cut_lengths = np.interp(dist,dist2,lengths2[::2])

# figure();plot(dist,cut_lengths)
# plot(dist2,lengths2[::2],'r')

In [217]:
# Now I just have to find out what the attenuation coefficients are and then I can make an array that has them
# In it to just multiply the final output by.

In [218]:
import fastcat.fastcat as fc
original_energies_keV = np.array([10,20,30, 40, 50 ,60, 70, 80 ,90 ,100 ,300 ,500 ,700, 900, 1000 ,2000 ,4000 ,6000])

In [219]:
lengths2 = lengths 

bowtie_filter = np.zeros((len(original_energies_keV),lengths2.shape[0]))
bowtie_filter2 = np.zeros((len(original_energies_keV),lengths2.shape[0]))

al_map = fc.get_mu(z=14)

for ii in range(len(original_energies_keV)):
    
    bowtie_filter[ii,:] = np.e**(-lengths2*al_map(original_energies_keV[ii]))
    bowtie_filter2[ii,:] = lengths2*al_map(original_energies_keV[ii])
#     print(al_map(original_energies_keV[ii]))

In [220]:
plt.figure()
plt.plot(bowtie_filter.T);
plt.legend(original_energies_keV)
plt.xlabel('pixel')
plt.ylabel('$I/I_0$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$I/I_0$')

In [221]:
plt.figure()
plt.plot(bowtie_filter2.T[::2,:]);

bowtie_smaller = bowtie_filter.T[::2,:]
bowtie_smaller2 = bowtie_filter2.T[::2,:]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [222]:
np.save('bowtie3',bowtie_smaller)
np.save('bowtie3_lengths',bowtie_smaller2)

In [201]:
# np.save('bowtie9',bowtie_filter.T)
# np.save('bowtie9_lengths',bowtie_filter2.T)

In [202]:
!mv bowtie* ../../fastcat/data/filters/

In [24]:
class IndexTracker(object):
    def __init__(self, ax, X):
        self.ax = ax
        ax.set_title('use scroll wheel to navigate images')

        self.X = X
        rows, cols, self.slices = X.shape
        self.ind = 10 #self.slices//2

        self.im = ax.imshow(self.X[:, :, self.ind],cmap='bone')
        self.update()

    def onscroll(self, event):
        print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = (self.ind + 1) % self.slices
        else:
            self.ind = (self.ind - 1) % self.slices
        self.update()

    def update(self):
        self.im.set_data(self.X[:, :, self.ind])
        ax.set_ylabel('slice %s' % self.ind)
        self.im.axes.figure.canvas.draw()

fig, ax = plt.subplots(1, 1)

# X = np.array(projections.T[:,:,5:])
X = bowtie_filter.T

tracker = IndexTracker(ax, X)

fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: not enough values to unpack (expected 3, got 2)

# Now lets try to make the flattening filter at a different distance

In [20]:
theta = math.atan(20/150) 

In [21]:
h = math.tan(theta)*30

In [22]:
print(h)

4.0


In [23]:
# So the whole thing should be about 8 cm if it is at a depth of 30 cm

In [27]:
figure();plot(lengths[::16])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
lengths_small = lengths[::16]

print(lengths_small)

[1.8102956  1.8081605  1.8027607  1.801784   1.7953762  1.7889802
 1.778515   1.7596264  1.7287586  1.6916375  1.6405048  1.5691274
 1.4293513  1.1626906  0.9428454  0.78842276 0.66576225 0.56037635
 0.47512865 0.39989367 0.33571047 0.27865955 0.23026493 0.1892935
 0.15276453 0.12075748 0.09490971 0.07204794 0.05557918 0.0429252
 0.03340923 0.03031756 0.03127966 0.03590642 0.04718997 0.06070281
 0.08167531 0.10240576 0.1309255  0.16372308 0.20298778 0.2451347
 0.29695696 0.35479975 0.4244624  0.49965295 0.5889652  0.693369
 0.82158846 0.98529536 1.2199944  1.4597743  1.577745   1.6469797
 1.6940248  1.7308096  1.7609013  1.7829177  1.7966682  1.8079659
 1.8156284  1.8219752  1.8307741  1.8433745 ]


In [32]:
print(np.linspace(0,8,64))

[0.         0.12698413 0.25396825 0.38095238 0.50793651 0.63492063
 0.76190476 0.88888889 1.01587302 1.14285714 1.26984127 1.3968254
 1.52380952 1.65079365 1.77777778 1.9047619  2.03174603 2.15873016
 2.28571429 2.41269841 2.53968254 2.66666667 2.79365079 2.92063492
 3.04761905 3.17460317 3.3015873  3.42857143 3.55555556 3.68253968
 3.80952381 3.93650794 4.06349206 4.19047619 4.31746032 4.44444444
 4.57142857 4.6984127  4.82539683 4.95238095 5.07936508 5.20634921
 5.33333333 5.46031746 5.58730159 5.71428571 5.84126984 5.96825397
 6.0952381  6.22222222 6.34920635 6.47619048 6.6031746  6.73015873
 6.85714286 6.98412698 7.11111111 7.23809524 7.36507937 7.49206349
 7.61904762 7.74603175 7.87301587 8.        ]


In [30]:
len(lengths_small)

64

In [33]:
np.linspace(0,8,64)

array([0.        , 0.12698413, 0.25396825, 0.38095238, 0.50793651,
       0.63492063, 0.76190476, 0.88888889, 1.01587302, 1.14285714,
       1.26984127, 1.3968254 , 1.52380952, 1.65079365, 1.77777778,
       1.9047619 , 2.03174603, 2.15873016, 2.28571429, 2.41269841,
       2.53968254, 2.66666667, 2.79365079, 2.92063492, 3.04761905,
       3.17460317, 3.3015873 , 3.42857143, 3.55555556, 3.68253968,
       3.80952381, 3.93650794, 4.06349206, 4.19047619, 4.31746032,
       4.44444444, 4.57142857, 4.6984127 , 4.82539683, 4.95238095,
       5.07936508, 5.20634921, 5.33333333, 5.46031746, 5.58730159,
       5.71428571, 5.84126984, 5.96825397, 6.0952381 , 6.22222222,
       6.34920635, 6.47619048, 6.6031746 , 6.73015873, 6.85714286,
       6.98412698, 7.11111111, 7.23809524, 7.36507937, 7.49206349,
       7.61904762, 7.74603175, 7.87301587, 8.        ])

In [ ]:
0.          0.12698413  0.25396825  0.38095238  0.50793651 0.63492063  0.76190476  0.88888889  1.01587302  1.14285714 1.26984127  1.3968254   1.52380952  1.65079365  1.77777778 1.9047619   2.03174603  2.15873016  2.28571429  2.41269841 2.53968254  2.66666667  2.79365079  2.92063492  3.04761905 3.17460317  3.3015873   3.42857143  3.55555556  3.68253968 3.80952381  3.93650794  4.06349206  4.19047619  4.31746032 4.44444444  4.57142857  4.6984127   4.82539683  4.95238095 5.07936508  5.20634921  5.33333333  5.46031746  5.58730159 5.71428571  5.84126984  5.96825397  6.0952381   6.22222222 6.34920635  6.47619048  6.6031746   6.73015873  6.85714286 6.98412698  7.11111111  7.23809524  7.36507937  7.49206349 7.61904762  7.74603175  7.87301587  8.